In [ ]:
import polars as pl


In [72]:
df = pl.read_parquet("/content/consolidado.parquet")

In [73]:
#Removendo colunas
df = df.drop(["start_station_name", "end_station_name"] )

In [74]:
# Conta valores nulos em cada coluna
df.null_count()

rideable_type,started_at,ended_at,member_casual
u32,u32,u32,u32
0,0,0,0


In [75]:
df.head()

rideable_type,started_at,ended_at,member_casual
str,str,str,str
"""electric_bike""","""2024-10-14 03:26:04.083""","""2024-10-14 03:32:56.535""","""member"""
"""electric_bike""","""2024-10-13 19:33:38.926""","""2024-10-13 19:39:04.490""","""member"""
"""electric_bike""","""2024-10-13 23:40:48.522""","""2024-10-13 23:48:02.339""","""member"""
"""electric_bike""","""2024-10-14 02:13:41.602""","""2024-10-14 02:25:40.057""","""member"""
"""electric_bike""","""2024-10-13 19:26:41.383""","""2024-10-13 19:28:18.560""","""member"""


In [80]:
#Definindo as colunas started_at e ended_at para tipo datas
df = df.with_columns(
    pl.col("started_at").str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S.%f"),
    pl.col("ended_at").str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S.%f"))

/tmp/ipython-input-1896794157.py:3: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  pl.col("started_at").str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S.%f"),
/tmp/ipython-input-1896794157.py:4: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  pl.col("ended_at").str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S.%f"))


In [88]:
# Extraindo as horas
df = df.with_columns(
    pl.col("started_at").dt.time().alias("start_hour"),
    pl.col("ended_at").dt.time().alias("end_hour")
)

In [93]:
#Extraindo o tempo de uso
df = df.with_columns(
    (pl.col("end_hour") - pl.col("start_hour")).alias("duration")
)

In [94]:
df.head()

rideable_type,started_at,ended_at,member_casual,start_hour,end_hour,duration
str,datetime[μs],datetime[μs],str,time,time,duration[ns]
"""electric_bike""",2024-10-14 03:26:04,2024-10-14 03:32:56,"""member""",03:26:04,03:32:56,6m 52s
"""electric_bike""",2024-10-13 19:33:38,2024-10-13 19:39:04,"""member""",19:33:38,19:39:04,5m 26s
"""electric_bike""",2024-10-13 23:40:48,2024-10-13 23:48:02,"""member""",23:40:48,23:48:02,7m 14s
"""electric_bike""",2024-10-14 02:13:41,2024-10-14 02:25:40,"""member""",02:13:41,02:25:40,11m 59s
"""electric_bike""",2024-10-13 19:26:41,2024-10-13 19:28:18,"""member""",19:26:41,19:28:18,1m 37s


In [100]:
# Extraindo os nomes dos dias e dos meses apatir da data
df = df.with_columns(
    day_name=pl.col("started_at").dt.strftime("%A"),
    month_name=pl.col("started_at").dt.strftime("%B"))

In [101]:
df.head()

rideable_type,started_at,ended_at,member_casual,start_hour,end_hour,duration,day_name,month_name
str,datetime[μs],datetime[μs],str,time,time,duration[ns],str,str
"""electric_bike""",2024-10-14 03:26:04,2024-10-14 03:32:56,"""member""",03:26:04,03:32:56,6m 52s,"""Monday""","""October"""
"""electric_bike""",2024-10-13 19:33:38,2024-10-13 19:39:04,"""member""",19:33:38,19:39:04,5m 26s,"""Sunday""","""October"""
"""electric_bike""",2024-10-13 23:40:48,2024-10-13 23:48:02,"""member""",23:40:48,23:48:02,7m 14s,"""Sunday""","""October"""
"""electric_bike""",2024-10-14 02:13:41,2024-10-14 02:25:40,"""member""",02:13:41,02:25:40,11m 59s,"""Monday""","""October"""
"""electric_bike""",2024-10-13 19:26:41,2024-10-13 19:28:18,"""member""",19:26:41,19:28:18,1m 37s,"""Sunday""","""October"""


In [105]:
# Removendo as horas, deixando somente as datas
df = df.with_columns (
    pl.col("started_at").dt.date(),
    pl.col("ended_at").dt.date()
)

In [108]:
# Salvando o arquivo tratado
df.write_parquet("consolidado_tratado.parquet")